# GNN

Dieses Notebook ist ein erster Test des GNN Ansatzes für die Vorhersage von $\mu$ und $\sigma$

## Import of Data

In [1]:
# Used to import the helper functions
import sys
sys.path.append('../utils')
from helpers import load_data, load_stations

In [2]:
data = load_data(indexed=False)
# Get List of stations with all -> will break further code if cut already
stations = load_stations(data)
# drop incomplete rows for fist test
data.drop(['sm_mean', 'sm_var'], inplace=True, axis=1)
# drop stations with elevation > 1000
data = data.drop(data[data['alt'] > 1000].index)

In [3]:
# Normalize the df
exclude_cols = ['date', 'obs', 'station','doy']

# normalize the DataFrame, excluding the columns to exclude
normalized_data = data.copy()  # create a copy of the original DataFrame to avoid modifying it directly
for col in data.columns:
    if col not in exclude_cols:
        normalized_data[col] = (data[col] - data[col].mean()) / data[col].std()

## Create the torch dataset

The Dataset which is a `pandas.DataFrame` gets converted to a `torch_geometric.data` object, which then can be processed by the GNN

#### ⚠️ TODO



In [4]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import geopy.distance
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
def dist_km(lat1, lon1, lat2, lon2):
    """Returns distance between two stations in km using the the WGS-84 ellipsoid.

    Args:
        lat1 (float): latitude of first station
        lat2 (float): latitude of second station
        lon1 (float): longitude of first station
        lon2 (float): longitude of second station

    Returns:
        float: distance in km
    """
    return geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km


def compute_dist_matrix(df: pd.DataFrame) -> np.array:
    """Returns a distance matrix between stations.

    Args:
        df (pd.DataFrame): dataframe with stations

    Returns:
        np.array: distance matrix
    """
    coords_df = df[['lat', 'lon']].copy()

    # create numpy arrays for latitudes and longitudes
    latitudes = np.array(coords_df['lat'])
    longitudes = np.array(coords_df['lon'])

    # create a meshgrid of latitudes and longitudes
    lat_mesh, lon_mesh = np.meshgrid(latitudes, longitudes)

    # calculate distance matrix using vectorized distance function
    distance_matrix = np.vectorize(dist_km)(lat_mesh, lon_mesh, lat_mesh.T, lon_mesh.T)
    return distance_matrix


def create_data(df: pd.DataFrame, date: pd.Timestamp, mask: np.array, dist_matrix: np.array) -> Data:
    """
    Create a PyTorch Geometric Data object from a given date.

    Args:
        df (pd.DataFrame): dataframe with stations and observations
        date (pd.Timestamp): date to create data for
        mask (np.array): Boolean array that represents the edges with distance less than max_dist
        dist_matrix (np.array): distance matrix between stations


    Returns:
        Data: PyTorch Geometric Data object
    """
    # Get the rows of the dataframe corresponding to the current date
    df = df[df['date'] == date]

    # Get the stations that reported for the current date
    reporting_stations = df['station'].to_numpy()
    assert any(df.station.value_counts() != 1) == False # Any Station that reports df should appear only once
    assert np.all(reporting_stations[:-1] < reporting_stations[1:]) # Array of Stations should be in ascending order

    # Create feature tensor (stations are orderd in the df)
    node_features = df.drop(['date', 'obs'], axis=1).to_numpy()
    x = torch.tensor(node_features, dtype=torch.float)
    
    # Create target tensor
    target = df['obs'].to_numpy()
    y = torch.tensor(target, dtype=torch.float)

    # Create a pairwise distance matrix (omit non reporting stations)
    mesh = np.ix_(reporting_stations, reporting_stations)
    mask_sliced = mask[mesh]

    # Get the indices of the edges to include
    edges = np.argwhere(mask_sliced)
    edges = edges.T

    edge_index = torch.tensor(edges, dtype=torch.int64)
    assert edge_index.shape[0] == 2
    
    #Create edge_attr tensor
    dist_matrix_sliced = dist_matrix[mesh]
    edge_attr = dist_matrix_sliced[edges[0], edges[1]]
    edge_attr = edge_attr.reshape(-1, 1)
    edge_attr = torch.tensor(edge_attr)
    # Create a PyTorch Geometric Data object
    df = Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr)
    return df

In [ ]:
#dist_matrix = compute_dist_matrix(stations)
#np.save(dist_matrix, 'dist_matrix.npy')

In [10]:
dist_matrix = np.load('dist_matrix.npy')

# TODO set the maximum distance (might want to get this from a configuration file)
max_dist = 50

# Create a boolean mask indicating which edges to include
mask = (dist_matrix <= max_dist) & (dist_matrix != 0)

In [11]:
torch_data = []

for date in tqdm(data['date'].unique()):
    torch_data.append(create_data(df=normalized_data, date=date, mask=mask, dist_matrix=dist_matrix))

100%|██████████| 3651/3651 [00:48<00:00, 75.56it/s]


### Dataloaders

In [8]:
# Definition of train_loader and valid_loader
# TODO another hyperparameter here
train_loader_small = DataLoader(torch_data[:365], batch_size=32, shuffle=True)
train_loader = DataLoader(torch_data[:-730], batch_size=32, shuffle=True)
valid_loader = DataLoader(torch_data[-730:-365], batch_size=32, shuffle=True)
test_data = torch_data[-365:]
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

### Visualizations of Graph to test if everything worked fine
Currently not working because data is scaled

In [9]:
import networkx as nx
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
%matplotlib inline
plt.style.use('default')

In [ ]:
def visualize_graph(d: Data):
    G = to_networkx(d, to_undirected=True)
    pos =  d.x[:,-5:-3].detach().numpy() # TODO this does not work if new features are added (add dict featureToIndex)
    pos = np.transpose([pos[:, 1], pos[:, 0]]) # Switch latitude and longitude
    # Rescale lat and long
    fac_lat = data["lat"].std()
    fac_lon = data["lon"].std()
    mean_lat = data["lat"].mean()
    mean_lon = data["lon"].mean()
    pos = pos * np.array([fac_lon, fac_lat]) + np.array([mean_lon, mean_lat])

    dict_pos = {i: p.tolist() for i, p in enumerate(pos)}

    # Plot map
    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.coastlines()
    ax.set_extent([5, 16, 47, 56], crs=proj)
    ax.add_feature(cfeature.LAND, color="lightgrey")
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.OCEAN)

    # Colors
    color_mask = np.any((d.edge_index.numpy().T[:, None] == np.array(G.edges)).all(axis=2),axis=1)
    color_values = d.edge_attr.flatten()[color_mask]

    cmap = mpl.colormaps.get_cmap('Greens_r')
    # Normalize the values to range between 0 and 1
    norm = plt.Normalize(min(color_values), max(color_values))
    # Generate a list of colors based on the normalized values
    colors = [cmap(norm(value)) for value in color_values]
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    plt.colorbar(sm, ax=ax)

    # Plot Graph
    nx.draw_networkx(G, pos=dict_pos, node_size=10, node_color="black", ax=ax, with_labels=False, edge_color=colors)

    # Fix the aspect ratio of the map
    lat_center = (ax.get_extent()[2] + ax.get_extent()[3]) / 2
    ax.set_aspect(1 / np.cos(np.radians(lat_center)))

    ax.set_title("Active weather stations in Germany")

    #plt.show()
    plt.savefig('plots/map_with_stations.png')

In [ ]:
visualize_graph(torch_data[1000])

## GNN

In the following section the gnn is constructed. This is largely based on the [PyG Colab Notebooks](https://colab.research.google.com/drive/1h3-vJGRVloF5zStxL5I0rSy4ZUPNsjy8?usp=sharing).

#### ⚠️ TODO
 - Modularize -> use multiple torch.modules in a Sequential model
 - Early stopping should save model checkpoint
 - gin config
 - [Neural Tangent Kernel](https://en.wikipedia.org/wiki/Neural_tangent_kernel)
 - Attention weights mitteln und anschauen (Kanten in Graph einfärben)
 - W&B hinzufügen
 - Check out GNN explainer
 - Hyperparameter Optimization
 - Test different Layers and Architectures
    - Embed Node ID (wie in paper von Sebastian)
    - ResNet (Recidual Layer)
    - Softplus for sigma
    - Edge Weights

### CRPS Loss Function

\begin{align*}

    \operatorname{crps}(F,X)=&\int_{-\inf}^{\inf}\left(F(y)-\boldsymbol{1}_{(y-x)}\right)^2dy\\

\end{align*}

Closed form expression from Gneiting et al. (2005)

\begin{align*}

    \operatorname{crps}\left(\mathcal{N}\left(\mu, \sigma^2\right), y\right)= & \sigma\left\{\frac{y-\mu}{\sigma}\left[2 \Phi\left(\frac{y-\mu}{\sigma}\right)-1\right] +2 \varphi\left(\frac{y-\mu}{\sigma}\right)-\frac{1}{\sqrt{\pi}}\right\}

\end{align*}

$\Phi\left(\frac{y-\mu}{\sigma}\right)$ denotes the CDF of a standard normal distribution and $\varphi\left(\frac{y-\mu}{\sigma}\right)$ denotes the PDF of a standard normal distribution

In [10]:
def crps(mu: torch.tensor, sigma: torch.tensor, y: torch.tensor):
    """Calculates the Continuous Ranked Probability Score (CRPS) assuming normally distributed df

    Args:
        mu (torch.tensor): mean
        sigma (torch.tensor): standard deviation
        y (torch.tensor): observed df

    Returns:
        torch.tensor: CRPS value
    """
    y = y.view((-1,1)) # make sure y has the right shape
    PI = np.pi #3.14159265359
    omega = (y - mu) / sigma
    # PDF of normal distribution at omega
    pdf = 1/(torch.sqrt(torch.tensor(2 * PI))) * torch.exp(-0.5 * omega ** 2)
    
    # Source: https://stats.stackexchange.com/questions/187828/how-are-the-error-function-and-standard-normal-distribution-function-related
    cdf = 0.5 * (1 + torch.erf(omega / torch.sqrt(torch.tensor(2))))
    
    crps = sigma * (omega * (2 * cdf - 1) + 2 * pdf - 1/torch.sqrt(torch.tensor(PI)))
    return  torch.mean(crps)

### GNN
Definition of Model and Training

In [11]:
from torch.nn import Linear, Embedding, Dropout, ModuleList
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, Sequential, summary
import torch_geometric

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
class Convolution(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_channels, heads, num_layers:int=None):
        super(Convolution, self).__init__()
        # Make sure either hidden_channels is a list, heads is a list or num_layer is supplied
        assert isinstance(hidden_channels, list) or isinstance(heads, list) or num_layers is not None,\
                "If hidden_channels is not a list, num_layers must be specified."
        # both are a list
        if isinstance(hidden_channels, list) and isinstance(heads, list):
            assert len(hidden_channels) == len(heads), f"Lengths of lists {len(hidden_channels)} and {len(heads)} do not match."
        # only hidden_channels is list
        if isinstance(hidden_channels, list) and not isinstance(heads, list):
            heads = [heads] * len(hidden_channels)
        # only heads is list
        if isinstance(heads, list) and not isinstance(hidden_channels, list):
            hidden_channels = [hidden_channels] * len(heads)
        # none is list
        if not isinstance(heads, list) and not isinstance(hidden_channels, list):
            heads = [heads] * num_layers
            hidden_channels = [hidden_channels] * num_layers
        # definition of Layers
        self.convolutions = ModuleList()
        for i, c, h in enumerate(zip(hidden_channels, heads)):
            if i == 0:
                # First Layer
                self.convolutions.append(GATConv(in_channels=input_dim, out_channels=c, heads=h))
            else:
                self.convolutions.append(GATConv(in_channels=-1, out_channels=c, heads=h))
        # Last Layer to match shape of output
        self.lin = Linear(in_features=hidden_channels[-1] * heads[-1], out_features=output_dim)

    def forward(self, x, edge_index):
        #TODO add edge_attrs here
        for i, conv in self.convolutions:
            x = F.relu(conv(x, edge_index))
        x = F.relu(self.lin(x))
        return x


class EmbedStations(torch.nn.Module):
    def __init__(self, num_stations_max, embedding_dim):
        super(EmbedStations, self).__init__()
        self.embed = Embedding(num_embeddings=num_stations_max, embedding_dim=embedding_dim)

    def forward(self, x):
        station_ids = x[:, 0].long()
        emb_station = self.embed(station_ids)
        x = torch.cat((emb_station, x[:, 1:]), dim=1) # Concatenate embedded station_id to rest of the feature vector
        return x


class MakePositive(torch.nn.Module):
    def __init__(self):
        super(MakePositive, self).__init__()

    def forward(self, x):
        mu, sigma = torch.split(x, 1, dim=-1)
        sigma = F.softplus(sigma) # ensure that sigma is positive
        return mu, sigma

In [15]:
class ResGnn(torch.nn.Module):
    def __init__(self, in_channels, out_channels, num_layers, hidden_channels, heads):
        super(ResGnn, self).__init__()
        assert num_layers > 0, "num_layers must be > 0."

        # Create Layers
        self.convolutions = ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.convolutions.append(GATConv(in_channels, hidden_channels, heads=heads))
            else:
                self.convolutions.append((GATConv(-1, hidden_channels, heads=heads)))
        self.lin = Linear(hidden_channels * heads, out_channels) #hier direkt 2 testen

    def forward(self, x, edge_index):
        #TODO add edge_attrs here
        for i, conv in enumerate(self.convolutions):
            if i == 0:
                # First Layer
                x = conv(x, edge_index)
                x = F.relu(x)
            else:
                x = x + F.relu(conv(x, edge_index)) # Residual Layers

        x = self.lin(x)
        x = F.relu(x)
        return x

In [16]:
class GNNOld(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, heads):
        super(GNNOld, self).__init__()
        #self.emb = Embedding(num_embeddings=531, embedding_dim=8)
        self.conv0 = GATConv(num_features, hidden_channels, heads=heads) #Layers mit edge attributes
        self.conv1 = GATConv(-1, hidden_channels, heads=heads)
        self.conv2 = GATConv(-1, hidden_channels, heads=heads)
        self.conv3 = GATConv(-1, hidden_channels, heads=heads)
        self.conv4 = GATConv(-1, hidden_channels, heads=heads)
        self.lin1 = torch.nn.Linear(hidden_channels * heads, 512) #hier direkt 2 testen
        self.lin2 = torch.nn.Linear(512, 2)

    def forward(self, x, edge_index):

        #emb_feature = self.emb(x[:, 0].long())
        #other_fetures = x[:, 1:]
        #x = torch.cat((emb_feature, other_features), dim=1)

        x = self.conv0(x, edge_index)
        x = F.relu(x)
        x = x + F.relu(self.conv1(x, edge_index)) # Resnet
        x = x + F.relu(self.conv2(x, edge_index)) # Resnet
        x = x + F.relu(self.conv3(x, edge_index)) # Resnet
        x = x + F.relu(self.conv4(x, edge_index)) # Resnet
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        mu, sigma = torch.split(x, 1, dim=-1)
        sigma = F.softplus(sigma) # ensure that sigma is positive
        return mu, sigma

In [21]:
# Clear Cache
torch.cuda.empty_cache()
# Initialize the model and optimizer
num_features = torch_data[0].num_features

#model = GNNOld(num_features=num_features, hidden_channels=16, heads=8)
model = Sequential('x, edge_index',
                   [
                       (EmbedStations(num_stations_max=531, embedding_dim=3), 'x -> x'),
                       (ResGnn(in_channels=-1, out_channels=512, hidden_channels=16, heads=8, num_layers=5), 'x, edge_index -> x'),
                       (Linear(512, 2),'x -> x'),
                       (MakePositive(), 'x -> mu, sigma')
                   ])
model.to(device)
torch_geometric.compile(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.002) #momentum anschauen

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
#TODO more hyperparams here
n_epochs = 400
patience = 30
best_val_loss = float('inf')
no_improvement = 0

def train(data):
    data.to(device)
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)#, df.edge_attr)
    mu, sigma = out
    loss = crps(mu, sigma, data.y)
    loss.backward()
    optimizer.step()
    return loss

train_losses = []
validation_losses = []

# loading bar
pbar = tqdm(range(n_epochs))

for epoch in pbar:
    # Train for one epoch
    model.train()
    train_loss = 0.0
    for data in train_loader:
        loss = train(data)
        train_loss += loss.item() * data.num_graphs
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
        
    # Evaluate on the validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            data.to(device)
            out = model(data.x, data.edge_index)#, df.edge_attr)
            mu, sigma = out
            loss = crps(mu, sigma, data.y)
            val_loss += loss.item() * data.num_graphs
        val_loss /= len(valid_loader.dataset)
        validation_losses.append(val_loss)

    # Check if the validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement = 0
        # Save model checkpoint
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, "checkpoint.pt")
    else:
        no_improvement += 1
    
    pbar.set_postfix({"Train Loss": train_loss, "Val Loss": val_loss,"Best Loss": best_val_loss, "No Improvement": no_improvement})
    # Early stopping
    if no_improvement == patience:
        print('Early stopping.')
        break

# Load weights from model checkpoint
checkpoint = torch.load("checkpoint.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# Set model to eval mode
model.eval()

  0%|          | 0/400 [00:00<?, ?it/s]
/opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [184,0,0], thread: [121,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [184,0,0], thread: [122,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [184,0,0], thread: [123,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [277,0,0], thread: [37,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [277,0,0], thread: [38,0,0] Assertion `srcI

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
print(summary(model, x=torch_data[1000].x.to(device), edge_index=torch_data[1000].edge_index.to(device)))#, edge_attr=None))

In [ ]:
plt.plot(train_losses[1:])
plt.plot(validation_losses[1:])

### Saving the Model
Every time the model is run a copy of the code and the model parameters are saved

In [ ]:
from datetime import datetime as dt
import shutil
import os

In [ ]:
# Save Model and losses
if False:
    date = str(dt.now()).replace(":","-").split(".")[0]
    path = f"modelsAndVariables/{date}"

    if not os.path.exists(path):
        os.makedirs(path)

    torch.save(model.state_dict(), f"{path}/model.pt")
    train_losses = np.array(train_losses)
    np.save(f"{path}/train_losses.npy", train_losses)
    validation_losses = np.array(validation_losses)
    np.save(f"{path}/test_losses.npy", validation_losses)

    dst_folder = path
    notebook_name = "gnn_test.ipynb"
    shutil.copy(notebook_name, path + "/" + notebook_name)

### Evaluation
Calculation of CRPS on test set and graphical diagnostics

In [ ]:
import numpy as np
model.eval()
mu_list = []
sigma_list = []
err_list = []
y_list = []


for data in test_loader:
    data.to(device)
    mu, sigma = model(data.x, data.edge_index)#, None)
    y = data.y
    err = crps(mu, sigma, y)
    mu = mu.detach().cpu().numpy().flatten()
    sigma = sigma.detach().cpu().numpy().flatten()
    y = y.cpu().numpy()
    err = err.detach().cpu().numpy()

    mu_list.append(mu)
    sigma_list.append(sigma)
    y_list.append(y)
    err_list.append(err*len(data))

err = sum(err_list) / len(test_loader.dataset)
mu = np.concatenate(mu_list)
sigma = np.concatenate(sigma_list)
y = np.concatenate(y_list)

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(np.arange(np.min(y),np.max(y),1), np.arange(np.min(y),np.max(y),1), linewidth=1, color="black")
plt.scatter(y, mu, c=sigma, s=1, linewidth=0, cmap="viridis")


cbar = plt.colorbar()
cbar.set_label('Sigma', rotation=270, labelpad=15)
plt.xlabel('observed temp.')
plt.ylabel('predicted temp.')
plt.title(f"Predicted vs. observed temperatures\nCRPS={err:.4f}")